In [1]:
import numpy as np
import scipy.sparse as sparse
from scipy.io import mmread
from scipy.linalg import expm, norm

In [2]:
dol = mmread('dolphins.mtx').tocsr()
dol

<62x62 sparse matrix of type '<class 'numpy.float64'>'
	with 318 stored elements in Compressed Sparse Row format>

In [3]:
# check symmetric

dol_array = sparse.csr_matrix.toarray(dol)
print(dol_array.transpose().all() == dol_array.all())

True


In [ ]:
# get dolphin names dictionary
dol_names = 'dolphins_nodename.txt'
f = open(dol_names, 'r') # read names text file

names = [] #create names list
for i in f:
    names.append(i.split()[0])
    
keys = list(range(len(names))) # create dictionary where keys are node number and vals are names
names_dict = dict(zip(keys,names))

In [8]:
import sys

print("The size in memory of the adjacency matrix is {0} MB".format(
    (sys.getsizeof(dol.shape)+
    dol.data.nbytes+
    dol.indices.nbytes+
    dol.indptr.nbytes)/(1024.0**2)
))

The size in memory of the adjacency matrix is 0.003940582275390625 MB


In [9]:
# Arnoldi algorithm

# matrix A for which exp(A)b is of interest, n x n

# b initial vector to be used, length n

# m, the produced Krylov subspace will have dimension m

def arnoldi_iteration(A, m: int): 
    b  = np.ones(A.shape[0])  # b default to be 1_n
    
    n = A.shape[0]
    h = np.zeros((m + 1, m)) # to become the m x m upper Hessenberg matrix consisting of the coefficients h_ij
    V = np.zeros((n, m + 1)) # to become the orthonormal basis V_m = [v_1, v_2, ..., v_m]
    v = b / norm(b) # makes v a unit 2-norm vector    
    V[:, 0] = v # use v as the first Krylov vector
    
    for j in range(m):
        w = A @ v  # compute candidate vector
        
        for i in range(j + 1):
            h[i, j] = V[:,i] @ w # h_ij-th element is product of v_i and w
            w = w - h[i, j] * V[:, i] # modified Gram-Schmidt
            
        h[j + 1, j] = norm(w)
        
        zero = 1e-12 # small value used as h_ij = 0 threshold
        if h[j + 1, j] > zero: # if nonzero add v to the basis
            v = w / h[j + 1, j]
            V[:, i + 1] = v
        else: 
            return V, h 
        # print('step',j,'out of',m) # to check how far along algorithm is for larger m
    return V, h

In [10]:
# Arnoldi approximation

# exp(A)v ~ beta x V_m x exp(H_m) x e_1 (m-space 1st unit vector [1, 0, 0,..., 0])

In [11]:
# get results 

m = 100

Vm1, Hm1 = arnoldi_iteration(dol, m)

Vm, Hm = sparse.csc_matrix(Vm1[:,0:m]), sparse.csc_matrix(Hm1[0:m,0:m])

In [12]:
# checks
zero = dol @ Vm - Vm @ Hm
print("check Arnoldi relation", zero.todense()) # should be zero matrix 
orthog = Vm.transpose() @ Vm
print("check orthogonality", orthog.todense()) # should be identity 

check Arnoldi relation [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.77555756e-17 -5.55111512e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.55111512e-17  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00 -1.38777878e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.77555756e-17  2.77555756e-17  4.16333634e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
check orthogonality [[ 1.00000000e+00  1.73472348e-18 -3.00107161e-16 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.73472348e-18  1.00000000e+00 -1.83880688e-16 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.00107161e-16 -1.83880688e-16  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 

In [13]:
def a_approximation(A, Vm, Hm, m):
    # get beta = ||v||_2
    b = np.ones(A.shape[0])
    beta = norm(b)

    # get unit vector
    e1 = np.zeros((m,1))
    e1[0] = 1
    e1 = sparse.csc_matrix(e1)
    
    X = beta * Vm @ expm(Hm) @ e1
    return X

In [14]:
expab = a_approximation(dol, Vm, Hm, m)

In [15]:
# max
expab.argmax()

14

In [16]:
# min
expab.argmin()

60

In [17]:
print('Loneliest dolphin is',names_dict[60])

Loneliest dolphin is Zig


In [18]:
print('Most popular dolphin is',names_dict[14])

Most popular dolphin is Grin
